In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras as ks
import prime as pr

primesh = pr.prime() #Create a prime object#Here we construct a single-point input for PRIME-SH.
#This line expects primesh.h5, and all scaler .pkl files to be in the same directory as this notebook.


In [ ]:
#PRIME-SH expects timeseries as input, so we need to construct a timeseries of length "window" with the same input values.

window = primesh.window #Length of timeseries needed for PRIME-SH
stride = primesh.stride #Stride of the timeseries
test_input = pd.DataFrame(columns = primesh.in_keys) #Initialize single-point input dataframe
test_input['B_xgsm'] = 0*np.ones(window) #SW BX in nT (GSM coordinates)
test_input['B_ygsm'] = 0*np.ones(window) #SW BY in nT (GSM coordinates)
test_input['B_zgsm'] = -5*np.ones(window) #SW BZ in nT (GSM coordinates)
test_input['Vi_xgse'] = -400*np.ones(window) #SW X velocity in km/s (GSE coordinates)
test_input['Vi_ygse'] = 0*np.ones(window) #SW Y velocity in km/s
test_input['Vi_zgse'] = 0*np.ones(window) #SW Z velocity in km/s
test_input['Ni'] = 5*np.ones(window) #SW density in cm^-3
test_input['Vth'] = 30*np.ones(window) #SW thermal velocity in km/s
test_input['R_xgse'] = 200*np.ones(window) #Wind position in X GSE (RE)
test_input['R_ygse'] = 0*np.ones(window) #Wind position in Y GSE
test_input['R_zgse'] = 0*np.ones(window) #Wind position in Z GSE

#This is where the location you're propagating to is set.
#Each entry in the "timeseries" is set to the same value, no need to make a fake MMS orbit.
test_input['target_R_xgse'] = 14*np.ones(window) #MMS position in X GSE (RE)
test_input['target_R_ygse'] = 0*np.ones(window) #MMS position in Y GSE
test_input['target_R_zgse'] = 0*np.ones(window) #MMS position in Z GSE

#PRIME-SH also currently expects an 'Epoch' for its inputs. This is helpful for ordering the timeseries, but makes little sense for a single point like we're doing here.
#Here we concoct a window of time that results in a prediction at UNIX timestamp 0.
test_input['Epoch'] = pd.date_range(start='1970-01-01 00:00:00', periods=window, freq='100s') - pd.Timedelta(hours = 2)

#Make one prediction. Predictions are returned as pandas dataframes.
primesh.predict(test_input)


In [ ]:
#Here we construct a grid of inputs for PRIME-SH.
gridsize = 0.5 #Size of the grid to calculate the magnetosheath conditions on (RE)
x_extent = [0,20] #Set the GSE X extent of the grid
y_extent = [-30,30] #Set the GSE Y extent of the grid

#This line will calculate the magnetosheath conditions at each point in the grid for one set of solar wind inputs.
sheath_grid_single = primesh.predict_grid(gridsize = gridsize, x_extent = x_extent, y_extent = y_extent, framenum = 1,
                                   bx = 0, by = 0, bz = -5, vx = -400, vy = 0, vz = 0, ni = 5, vt = 30,
                                   rx = 200, ry = 0, rz = 0, loc_mask = True, subtract_ecliptic=True)

#For a range of solar wind inputs, specify the inputs as a list, and set framenum to the length of the list.
bz = [-5, 5]
framenum = len(bz)
bx = 0*np.ones(framenum) #Currently, bx, by, bz, and ni must be the same size if specified as a list
by = 0*np.ones(framenum)
ni = 5*np.ones(framenum)
sheath_grid_range = primesh.predict_grid(gridsize = gridsize, x_extent = x_extent, y_extent = y_extent, framenum = framenum,
                                   bx = 0, by = 0, bz = bz, vx = -400, vy = 0, vz = 0, ni = ni, vt = 30,
                                   rx = 200, ry = 0, rz = 0, loc_mask = True, subtract_ecliptic=True)

#Each sheath_grid has shape [framenum, x_grid, y_grid, primesh.tar_keys]
